###1. Set file paths

In [0]:
from pyspark.sql.functions import expr

departure_delays_file_path = "dbfs:/FileStore/shared_uploads/sergio.salgado@n.world/departuredelays-1.csv"
airport_codes_file_path = "dbfs:/FileStore/shared_uploads/sergio.salgado@n.world/airport_codes_na-1.txt"

###2. Obtain the datasets

#####2.1 Obtain the departure delays dataset

In [0]:
departure_delays = (spark
                    .read
                    .format("csv")
                    .options(header = "true")
                    .load(departure_delays_file_path)
                   )

departure_delays = (departure_delays
                    .withColumn("delay", expr("CAST(delay as INT) as delay"))
                    .withColumn("distance", expr("CAST(distance as INT) as distance"))
                   )

departure_delays.createOrReplaceTempView("departure_delays")

#####2.2 Obtain the airports dataset

In [0]:
airports = (spark
            .read
            .format("csv")
            .options(header="true", inferSchema= "true", sep= "\t")
            .load(airport_codes_file_path)
             )

airports.createOrReplaceTempView("airports")

###3. Create a temporary small table

In [0]:
table_temp = (departure_delays
              .filter(expr("""origin == 'SEA' and destination == 'SFO' and date like '01010%' and delay > 0"""))
             )
table_temp.createOrReplaceTempView("table_temp")

###4. Show the datasets

#####4.1 Show the airports dataset

In [0]:
spark.sql("SELECT * FROM airports LIMIT 10").show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
+-----------+-----+-------+----+



#####4.2 Show the departure delays dataset

In [0]:
spark.sql("SELECT * FROM departure_delays LIMIT 10").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+



#####4.3 Show the temporary table

In [0]:
spark.sql("SELECT * FROM table_temp LIMIT 10").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



###5. Unions

#####5.1 Union two tables

In [0]:
bar = departure_delays.union(table_temp)
bar.createOrReplaceTempView("bar")

#####5.2 Show the union

In [0]:
# filtering for SEA and SFO in a specifict time range
bar.filter(expr("""origin == 'SEA' AND destination == 'SFO' AND date LIKE '01010%' AND delay > 0""")).show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



###6. Joins

In [0]:
# Join departure delays data (foo) with airport info
table_temp.join(
    airports,
    airports.IATA == table_temp.origin
).select("City", "State", "date", "delay", "distance", "destination").show()

+-------+-----+--------+-----+--------+-----------+
|   City|State|    date|delay|distance|destination|
+-------+-----+--------+-----+--------+-----------+
|Seattle|   WA|01010710|   31|     590|        SFO|
|Seattle|   WA|01010955|  104|     590|        SFO|
|Seattle|   WA|01010730|    5|     590|        SFO|
+-------+-----+--------+-----+--------+-----------+



###7. Windowing

#####7.1 Total review of delays experienced in flights from Seattle(SEA), San Francisco(SFO) and New York City(JFK)

In [0]:
%sql
DROP TABLE IF EXISTS departure_delays_window;

CREATE TABLE departure_delays_window AS
SELECT origin, destination, SUM(delay) AS TotalDelays
FROM departure_delays
WHERE origin IN ('SEA', 'SFO', 'JFK')
AND destination IN ('SEA', 'SFO', 'JFK', 'DEN', 'ORD', 'LAX', 'ATL')
GROUP BY origin, destination;

SELECT * FROM departure_delays_window

origin,destination,TotalDelays
JFK,ORD,5608
JFK,SFO,35619
JFK,DEN,4315
JFK,ATL,12141
JFK,SEA,7856
JFK,LAX,35755
SEA,LAX,9359
SFO,ORD,27412
SFO,DEN,18688
SFO,SEA,17080


#####7.2 Find the three destinations that experienced the most delays with those three ariports as origin

In [0]:
%sql
SELECT origin, destination, SUM(TotalDelays) AS TotalDelays
FROM departure_delays_window
WHERE origin = '[ORIGIN]'
GROUP BY origin, destination
ORDER BY SUM(TotalDelays) DESC
LIMIT 3


com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: Resolved attribute(s) TotalDelays#4064L missing from origin#4068,destination#4069,TotalDelays#4070L in operator !Aggregate [origin#4068, destination#4069], [origin#4068, destination#4069, sum(TotalDelays#4070L) AS TotalDelays#4064L, sum(TotalDelays#4064L) AS sum(TotalDelays#4064L)#4074L]. Attribute(s) with the same name appear in the operation: TotalDelays. Please check if the right attribute(s) are used.;
GlobalLimit 3
+- LocalLimit 3
 +- Project [origin#4068, destination#4069, TotalDelays#4064L]
 +- Sort [sum(TotalDelays#4064L)#4074L DESC NULLS LAST], true
 +- !Aggregate [origin#4068, destination#4069], [origin#4068, destination#4069, sum(TotalDelays#4070L) AS TotalDelays#4064L, sum(TotalDelays#4064L) AS sum(TotalDelays#4064L)#4074L]
 +- Filter (origin#4068 = [ORIGIN])
 +- SubqueryAlias spark_catalog.default.departure_delays_window
 +- Relation default.departure_delays_window[origin#4068,destination#4069,TotalDelays#4070L] parquet

	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.failAnalysis(CheckAnalysis.scala:60)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.failAnalysis$(CheckAnalysis.scala:59)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.failAnalysis(Analyzer.scala:222)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$2(CheckAnalysis.scala:533)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$2$adapted(CheckAnalysis.scala:105)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:358)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:357)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:357)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:357)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:357)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(Iter

#####7.3 Do the same as before but with the dense_rank() window function

In [0]:
%sql
spark.sql("""
SELECT origin, destination, TotalDelays, rank
  FROM (
    SELECT origin, destination, TotalDelays, dense_rank()
      OVER (PARTITION BY origin ORDER BY TotalDelays DESC) as rank
      FROM departure_delays_window
  ) t
  WHERE rank <= 3
""").show()


com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.catalyst.parser.ParseException: 
mismatched input 'spark' expecting {'(', 'APPLY', 'CONVERT', 'COPY', 'OPTIMIZE', 'RESTORE', 'ADD', 'ALTER', 'ANALYZE', 'CACHE', 'CLEAR', 'COMMENT', 'COMMIT', 'CREATE', 'DELETE', 'DESC', 'DESCRIBE', 'DFS', 'DROP', 'EXPLAIN', 'EXPORT', 'FROM', 'GRANT', 'IMPORT', 'INSERT', 'LIST', 'LOAD', 'LOCK', 'MAP', 'MERGE', 'MSCK', 'REDUCE', 'REFRESH', 'REPLACE', 'RESET', 'REVOKE', 'ROLLBACK', 'SELECT', 'SET', 'SHOW', 'START', 'SYNC', 'TABLE', 'TRUNCATE', 'UNCACHE', 'UNLOCK', 'UPDATE', 'USE', 'VALUES', 'WITH'}(line 1, pos 0)

== SQL ==
spark.sql("""
^^^
SELECT origin, destination, TotalDelays, rank
 FROM (
 SELECT origin, destination, TotalDelays, dense_rank()
 OVER (PARTITION BY origin ORDER BY TotalDelays DESC) as rank
 FROM departure_delays_window
 ) t
 WHERE rank <= 3
""").show()

	at org.apache.spark.sql.catalyst.parser.ParseException.withCommand(ParseDriver.scala:298)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parse(ParseDriver.scala:159)
	at org.apache.spark.sql.execution.SparkSqlParser.parse(SparkSqlParser.scala:88)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parsePlan(ParseDriver.scala:106)
	at com.databricks.sql.parser.DatabricksSqlParser.$anonfun$parsePlan$1(DatabricksSqlParser.scala:77)
	at com.databricks.sql.parser.DatabricksSqlParser.parse(DatabricksSqlParser.scala:97)
	at com.databricks.sql.parser.DatabricksSqlParser.parsePlan(DatabricksSqlParser.scala:74)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$2(SparkSession.scala:801)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:268)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$1(SparkSession.scala:801)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:968)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:798)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:695)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.$anonfun$executeSql$1(SQLDriverLocal.scala:91)
	at scala.collection.immutable.List.map(List.scala:293)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:37)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:145)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$12(DriverLocal.scala:631)
	at com.databricks.logging.Log4jUsageLoggingShim$.$anonfun$withAttributionContext$1(Log4jUsageLoggingShim.scala:33)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:94)
	at com.databricks.logging.Log4jUsageLoggingShim$.withAttributionContext(Log4jUsageLoggingShim.scala:31)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:205)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:204)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:59)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:240)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:225)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:59)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:608)
	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:615)
	at scala.util.Try$.apply(Try.scala:213)
	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:607)
	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommandAndGetError(DriverWrapper.scala:526)
	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:561)
	at com.databricks.backend.daemon.driver.Driv

###8. Modifications

#####8.1 Select the table_temp dataset

In [0]:
table_temp.show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



#####8.2 Add a new column

In [0]:
from pyspark.sql.functions import expr

table_temp_2 = (table_temp
                .withColumn("satus", expr("CASE WHEN delay <= 10 THEN 'On-time' ELSE 'Delayed' 'END'"))
               )

---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
<command-374121175550857> in <module>
      2 
      3 table_temp_2 = (table_temp
----> 4                 .withColumn("satus", expr("CASE WHEN delay <= 10 THEN 'On-time' ELSE 'Delayed' 'END'"))
      5                )

/databricks/spark/python/pyspark/sql/functions.py in expr(str)
   1515     """
   1516     sc = SparkContext._active_spark_context
-> 1517     return Column(sc._jvm.functions.expr(str))
   1518 
   1519 

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
    121                 # Hide where the exception 

#####8.3 Drop the delay column

In [0]:
table_temp_3 = table_temp_2.drop("delay")
table_temp_3.show()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-374121175550858> in <module>
----> 1 table_temp_3 = table_temp_2.drop("delay")
      2 table_temp_3.show()

NameError: name 'table_temp_2' is not defined

#####8.4 Rename the column "status"

In [0]:
table_temp_4 = table_temp_3.withColumnRenamed("status", "flight_status")
table_temp_4.show()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-374121175550855> in <module>
----> 1 table_temp_4 = table_temp_3.withColumnRenamed("status", "flight_status")
      2 table_temp_4.show()

NameError: name 'table_temp_3' is not defined

###9. Pivoting

In [0]:
%sql
-- swap the columns for the rows

SELECT destination, CAST(SUBSTRING(date, 0, 2) AS int) AS month, delay
  FROM departure_delays
  WHERE origin = 'SEA'

destination,month,delay
ORD,1,92
JFK,1,-7
DFW,1,-5
MIA,1,-3
DFW,1,-3
DFW,1,1
ORD,1,-10
DFW,1,-6
DFW,1,-2
ORD,1,-3


In [0]:
%sql
-- pivoting to place names in the month column and aggregate calculations on the delays by destination and month

SELECT * FROM (
SELECT destination, CAST(SUBSTRING(date, 0, 2)AS int) AS month, delay
  FROM departure_delays  WHERE origin = 'SEA'
)
PIVOT (
  CAST(AVG(delay) AS DECIMAL(4, 2)) AS AvgDelay, MAX(delay) AS MaxDelay
  FOR month IN (1 JAN, 2 FEB)
)
ORDER BY destination

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: Table or view not found: departure_delays; line 3 pos 7;
'Sort ['destination ASC NULLS FIRST], true
+- 'Project [*]
 +- 'Pivot 'month, [1, 2], [cast('AVG('delay) as decimal(4,2)), 'MAX('delay)]
 +- 'SubqueryAlias __auto_generated_subquery_name
 +- 'Project ['destination, cast('SUBSTRING('date, 0, 2) as int) AS month#8, 'delay]
 +- 'Filter ('origin = SEA)
 +- 'UnresolvedRelation [departure_delays], [], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$2(CheckAnalysis.scala:138)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$2$adapted(CheckAnalysis.scala:105)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:358)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:357)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:357)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:357)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:357)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:357)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(Ite